**В этом ноутбуке я провожу оценку производительности ранее обученной модели   
(и всей рек.системы в целом) на отложенных оригинальных тестовых данных test_main**

**Какие данные используются**

Я не буду делать предсказания на всем test_main, чтобы затем загрузить submission на kaggle.  
Я как раньше, разделю **test_main** на 2 равные половины (по хронологии aid в каждой сессии) -   
1. на тест.часть **test_history_sessions**
2.  на тестовые метки **test_labels**

**Я не буду переобучать модель, полученную на меньшой части от train_main (на train_third_w)  
на всем train_main. Это займет много времени, поэтому я попробую получить предсказания на test_main  
с помощью обученной ранее модели CatBoost Ranker**

В итоге я проверю производительность модели локально на test_labels, а не на тех тестовых метках,  
находящихся на LB на Kaggle

In [ ]:
# main_path = '/Users/stanislavkrupnov/Jup.Notebook'

main_path = '/content/drive/Othercomputers/Mac/Jup.Notebook'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [ ]:
import os
import sys
from tqdm import tqdm
import glob
import pickle
import pandas as pd
import gc
import polars as pl
import numpy as np
from itertools import product
from collections import Counter, defaultdict
import pyarrow.parquet as pq
import shutil
import gdown
import dask
import dask.dataframe as dd

# Функции

In [ ]:
def metric_eval(predictions: pd.DataFrame, valid: pd.DataFrame,
                id2type: dict) -> tuple:
    """
    Вычисляет метрики для оценки качества модели.

    Parameters:
    - predictions (pd.DataFrame): DataFrame с предсказаниями модели.
    - valid (pd.DataFrame): DataFrame с данными для валидации.
    - id2type (dict): Словарь для преобразования индексов типов в соответствующие строки.

    Returns:
    - tuple: Кортеж, содержащий локальную метрику и реколл для каждого типа.
    """
    # Преобразование индексов типов в соответствующие строки
    valid.type = valid.type.map(lambda idx: id2type[idx])

    # Создание списка с метками для каждой сессии и типа
    ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
    ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})

    # Обрезка меток для типа 'clicks' до одной метки
    ground_truth.loc[ground_truth.type == 'clicks',
                     'labels'] = ground_truth.loc[ground_truth.type ==
                                                  'clicks', 'labels'].str[:1]

    # Объединение предсказаний с истинными метками
    submission_with_gt = predictions.merge(
        ground_truth[['session', 'type', 'labels']],
        how='left',
        on=['session', 'type'])

    # Отбрасывание сессий без истинных меток
    submission_with_gt = submission_with_gt[~submission_with_gt.labels_y.isna(
    )]

    # Вычисление количества совпадений между предсказанными и истинными метками
    submission_with_gt['hits'] = submission_with_gt.apply(
        lambda df: len(set(df.labels_x).intersection(set(df.labels_y))),
        axis=1)

    # Вычисление количества истинных меток для каждого типа
    submission_with_gt['gt_count'] = submission_with_gt.labels_y.str.len(
    ).clip(0, 20)

    # Вычисление реколла для каждого типа
    recall_per_type = submission_with_gt.groupby([
        'type'
    ])['hits'].sum() / submission_with_gt.groupby(['type'])['gt_count'].sum()

    # Вычисление локальной метрики на основе реколла для каждого типа
    local_validation_score = (recall_per_type * pd.Series({
        'clicks': 0.10,
        'carts': 0.30,
        'orders': 0.60
    })).sum()

    return local_validation_score, recall_per_type

In [ ]:
# Функция для чтения данных из кеша
def read_file(f):
    # Возвращаем DataFrame, соответствующий ключу f в словаре data_cache
    return cudf.DataFrame( data_cache[f] )

In [ ]:
def read_covis_to_dict(name: str, n: int) -> dict:
    '''
    Считывает данные из нескольких файлов, содержащих матрицу совстречаемости,
    и объединяет их в словарь, где ключи - aid_x, значения - списки связанных aid_y.

    Параметры:
    - name (str): Имя матрицы, используется для формирования пути к файлам.
    - n (int): Количество файлов с матрицей.

    Возвращает:
    - covis_matrix (Dict[str, List[str]]): Словарь, где ключи - aid_x, значения - списки связанных aid_y.
    '''
    # Инициализация пустого словаря для хранения матрицы совстречаемости
    covis_matrix = {}

    # Чтение данных из файлов и обновление словаря
    for k in tqdm(range(0, n)):
        path = f'{main_path}/LB/covis/{name}/{name}_{k}.parquet'

        # Первый файл создает словарь, последующие обновляют его
        if k == 0:
            covis_matrix = pd.read_parquet(path)
            covis_matrix = covis_matrix.groupby('aid_x').aid_y.apply(
                list).to_dict()
        else:
            df = pd.read_parquet(path)
            df = df.groupby('aid_x').aid_y.apply(list).to_dict()
            covis_matrix.update(df)

    return covis_matrix

In [ ]:
def read_parquet_in_chunks(file_path: str, batch_num: int):
    '''
    Читает Parquet-файл порциями и возвращает DataFrame.

    Parameters:
    - file_path (str): Путь к Parquet-файлу.
    - batch_num (int): Количество порций для чтения.

    Yields:
    - pd.DataFrame: DataFrame, представляющий порцию данных из Parquet-файла.
    '''
    table = pq.read_table(file_path)
    num_rows = len(table)
    batch_size = num_rows // batch_num

    for i in range(0, num_rows, batch_size):
        yield table[i:i + batch_size].to_pandas()

In [ ]:
def create_labels(n: int,
                  inp_dict: dict,
                  covis: dict,
                  top_pop_aids: set,
                  cov_f: bool,
                  b: int = -1) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений.

    Parameters:
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.
    - b (int): Номер файла для сохранения, если cov_f=True.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    featuress = {}

    for session, aids in tqdm(inp_dict.items()):
        # Инициализация словаря для подсчета aids
        aid_counter = Counter()

        # Цикл по всем aids в сессии
        for aid in aids:
            # Если aid есть в covis_matrix, увеличиваем его счетчик
            if aid in covis:
                aid_counter.update(covis[aid])

        if len(set(aids)) >= n:  # Оставлю только последние n уникальные aids
            labels[session] = list(dict.fromkeys(aids))[-n:]
        else:
            # Получаем топ встречающихся aids
            top_aids = [
                aid
                for aid, count in aid_counter.most_common(n - len(set(aids)))
            ]

            # Добавляем исходные aids и топ встречающихся aids в labels
            labels[session] = list(set(list(set(aids)) + list(set(top_aids))))

            if len(set(aids)) < n:
                pops = list(top_pop_aids.difference(set(labels[session])))
                labels[session] += pops[:n - len(labels[session])]

        if cov_f == False:
            continue
        else:
            # Создание словаря с признаком - рангом (по ко-виз матрице) каждого кандидата внутри сессии
            for aid in labels[session]:
                if aid in list(aid_counter.keys()):
                    if session not in featuress:
                        featuress[session] = {}
                    featuress[session][aid] = aid_counter[aid]

    if cov_f == True:
        if b == -1:  # Если мы в функции create_labels
            path_ = f'{main_path}/tr/{ver_folder}/featuress.pickle'
        else:  # Если мы в функции create_labels_in_batches
            path_ = f'{main_path}/tr/{ver_folder}/featuress_{b}.pickle'

        with open(path_, 'wb') as file:
            pickle.dump(featuress, file)

        print(
            f'Словарь с признаком-рангом по ковиз-матрице сохранен в : {path_}'
        )

    return labels

In [ ]:
def create_labels_in_batches(k: int, n: int, inp_dict: dict, covis: dict,
                             top_pop_aids: set, cov_f: bool) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений по батчам.

    Parameters:
    - k (int): Количество батчей.
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    partt = len(inp_dict) // k

    for batch in tqdm(range(k), desc='part'):

        start = partt * batch
        end = partt * (batch + 1)
        slicee_ = list(inp_dict.items())[start:end]
        slicee = dict(slicee_)

        labels_ = create_labels(n=n,
                                inp_dict=slicee,
                                covis=covis,
                                top_pop_aids=top_pop_aids,
                                cov_f=cov_f,
                                b=batch)
        labels.update(labels_)

    return labels

In [ ]:
def create_pop_features(for_features_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Создает признак ранжирования aid по встречаемости в группах по типам.

    Parameters:
    - for_features_df (pd.DataFrame): Датасет с данными для признака.

    Returns:
    - pd.DataFrame: Датасет с признаком.
    '''
    # Создадим пустой DataFrame, в который будем записывать результаты
    result_df = pd.DataFrame()

    # Сгруппируем исходный DataFrame по столбцу 'type'
    grouped = for_features_df.groupby('type')

    id_2_type = {0: 'clicks', 1: 'carts', 2: 'orders'}

    # Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
    for name, group in tqdm(grouped):
        # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
        aid_counts = group['aid'].value_counts().reset_index()
        aid_counts.columns = ['aid', 'count']

        # Добавим столбец с рангом, используя функцию rank()
        aid_counts['rank'] = aid_counts['count'].rank(ascending=False,
                                                      method='dense')
        aid_counts.drop(columns=['count'], inplace=True)

        # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
        merged_group = pd.merge(group, aid_counts, on='aid', how='left')

        # Создадим имена для столбцов рангов на основе типа 'type'
        rank_column_name = 'rank_' + id_2_type[name]
        merged_group.rename(columns={'rank': rank_column_name}, inplace=True)

        result_df = pd.concat([result_df, merged_group], ignore_index=True)

        result_df.fillna(-1, inplace=True)
        result_df.rank_clicks = result_df.rank_clicks.astype(np.int32)
        result_df.rank_carts = result_df.rank_carts.astype(np.int32)
        result_df.rank_orders = result_df.rank_orders.astype(np.int32)
        result_df.reset_index(inplace=True, drop=True)

    return result_df

In [ ]:
def add_targets(dataframe_: pd.DataFrame, targets: pd.DataFrame,
                targ_type: int) -> pd.DataFrame:
    '''
    Добавляет таргеты к датафрейму.

    Parameters:
    - dataframe_ (pd.DataFrame): Датафрейм, которому нужны таргеты.
    - targets (pd.DataFrame): Датафрейм с таргетами всех типов.
    - targ_type (int): Текущий тип таргета.

    Returns:
    - pd.DataFrame: Датафрейм с нужными таргетами.
    '''
    targets_ = targets.loc[targets['type'] == targ_type,
                           ['session', 'aid', 'type']].rename(columns={
                               'type': 'target'
                           }).copy()
    targets_['target'] = 1
    targets_ = targets_.drop_duplicates()
    merge_cols = ["session", "aid"]
    dataframe = dataframe_.merge(targets_, on=merge_cols, how="left")
    dataframe = dataframe.fillna(0)
    dataframe.target = dataframe.target.astype(np.uint8)

    return dataframe

In [ ]:
def w_pbar(pbar: tqdm, func: callable):
    '''
    Оборачивает функцию, обновляя tqdm прогресс бар.

    Parameters:
    - pbar (tqdm): Прогресс бар.
    - func (callable): Функция.

    Returns:
    - callable: Обернутая функция.
    '''
    def foo(*args, **kwargs):
        pbar.update(1)
        return func(*args, **kwargs)

    return foo

In [ ]:
def read_parquets(path: str) -> pd.DataFrame:
    '''
    Считывает данные из нескольких файлов формата Parquet, находящихся в заданной директории,
    и объединяет их в один DataFrame.

    Параметры:
    - path (str): Путь к файлам формата Parquet.

    Возвращает:
    - tr_candidates (pd.DataFrame): Объединенный DataFrame, содержащий данные из файлов Parquet.
    '''
    # Получение списка файлов Parquet в указанной директории
    file_list = glob.glob(path)

    # Инициализация пустого DataFrame для сбора данных
    tr_candidates = pd.DataFrame()

    # Цикл для чтения и объединения файлов
    for file in tqdm(file_list):
        # Чтение файла Parquet во временный DataFrame
        df_ = pd.read_parquet(file)

        # Объединение временного DataFrame с основным датасетом
        tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

    return tr_candidates

# Разбиение данных

In [ ]:
test_main = pd.read_parquet(f'{main_path}/main_dataframes/test_main.parquet')

In [ ]:
def split_df_into_halves(df: pd.DataFrame) -> tuple:
    """
    Разделить датафрейм на две части для каждой сессии, основываясь на столбце 'ts'
    внутри каждой сессии.

    Параметры:
        df: Входной датафрейм с колонками 'session', 'aid' и 'ts'.

    Возвращает:
        tuple: Кортеж, содержащий два датафрейма. Первый датафрейм содержит
        первую половину 'aid' для каждой сессии, а второй датафрейм содержит вторую
        половину 'aid' для каждой сессии.
    """
    # Сортировка входного датафрейма по столбцу 'ts' внутри каждой сессии
    df.sort_values(by=['session', 'ts'], inplace=True)

    # Разделение датафрейма на две половины (первую и вторую половины 'aid')
    half_size = df.groupby('session')['aid'].transform('size') // 2
    first_half_df = df[
        df.groupby('session').cumcount() < half_size].reset_index(drop=True)
    second_half_df = df[
        df.groupby('session').cumcount() >= half_size].reset_index(drop=True)

    return first_half_df, second_half_df

In [ ]:
test_candidates_sessions, test_labels = split_df_into_halves(test_main)

In [ ]:
folder_path = f'{main_path}/LB/data'
os.makedirs(folder_path, exist_ok=True)

#  Сохраняю для дальнейшего использования
test_labels.to_parquet(f'{main_path}/LB/data/test_labels.parquet')
test_candidates_sessions.to_parquet(
    f'{main_path}/LB/data/test_candidates_sessions.parquet')

**Разбиение на батчи для создания co-vis matrix**

In [ ]:
folder_path = 'test_candidates_sessions_b'
os.makedirs(folder_path, exist_ok=True)

dask_df = dd.from_pandas(test_candidates_sessions, npartitions=2)
dask_df.repartition(20).to_parquet(f'{folder_path}/')

# Co-vis матрицы создание

In [ ]:
matrix_name = 'test_covis_matrix'

In [ ]:
# Чтобы работало - запускаю ячейку, затем перезагружаю сеанс(не отключаюсь), и еще раз
# - все робит!
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [ ]:
import cudf

In [ ]:
data_cache = {} # Кеш данных CPU
type_labels = {'clicks':0, 'carts':1, 'orders':2} # Словарь меток типов

In [ ]:
# Считывание и кеширование данных
files = glob.glob(f'test_candidates_sessions_b/*') # Получаем список файлов
for f in files: data_cache[f] = pd.read_parquet(f) #

In [ ]:
READ_CT = 5
CHUNK = int( np.ceil( len(files)/6 )) # Размер каждого chunk

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7dc40e578b30 to Device at 0x7dc40e56dcf0>

In [ ]:
%%time

# Создание директории для сохранения файлов
folder_path = f'{matrix_name}'
os.makedirs(folder_path, exist_ok=True)

# Веса для каждого типа события
type_weight = {0: 1, 1: 6, 2: 3}

# Количество частей для разделения данных (оптимизация использования памяти)
disk_pieces = 20

# Размер каждой части данных
size = 1.86e6 / disk_pieces

# Вычисление частями для управления памятью
for part in range(disk_pieces):
    print()
    print('### ЧАСТЬ НА ДИСКЕ', part + 1)

    # Слияние - самый быстрый процесс слияния частей внутри частей
    # => Внешние части
    for j in range(6):
        a = j * CHUNK
        b = min((j + 1) * CHUNK, len(files))
        print(f'Обработка файлов с {a} по {b-1} в группах по {READ_CT}...')

        tmp2 = None
        df = None

        # => Внутренние части
        for k in range(a, b, READ_CT):
            # Чтение файла
            df = [read_file(files[k])]
            for i in range(1, READ_CT):
                if k + i < b:
                    df.append(read_file(files[k+i]))
            df = cudf.concat(df, ignore_index=True, axis=0)
            df = df.sort_values(['session', 'ts'], ascending=[True, False])

            # Использование хвоста сессии
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n < 70].drop('n', axis=1)

            # Создание пар
            df = df.merge(df, on='session')
            df = df.loc[((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y)]

            # Управление памятью - вычисление частями
            df = df.loc[(df.aid_x >= part * size) & (df.aid_x < (part + 1) * size)]

            # Назначение весов
            # Применяет веса к типам событий и суммирует веса для каждой пары 'aid'.
            df = df[['session', 'aid_x', 'aid_y', 'type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = df.type_y.map(type_weight)
            df = df[['aid_x', 'aid_y', 'wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x', 'aid_y']).wgt.sum()

            # Сочетание внутренних частей
            if k == a:
               tmp2 = df
            else:
               tmp2 = tmp2.add(df, fill_value=0)
            print(k, ', ', end='')
        print()

        # Сочетание внешних частей
        if a == 0:
          tmp = tmp2
        else:
          tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()

    # Преобразование матрицы в словарь
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x', 'wgt'], ascending=[True, False])

    # Сохранение топ-50
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n < 50].drop('n', axis=1)

    # Сохранение части на диск (конвертация в pandas сначала использует меньше памяти)
    tmp.to_pandas().to_parquet(f'{matrix_name}/{matrix_name}_{PART}.parquet')



### DISK PART 1
Processing files 0 thru 3 in groups of 5...
0 , 
Processing files 4 thru 7 in groups of 5...
4 , 
Processing files 8 thru 11 in groups of 5...
8 , 
Processing files 12 thru 15 in groups of 5...
12 , 
Processing files 16 thru 19 in groups of 5...
16 , 
Processing files 20 thru 19 in groups of 5...


### DISK PART 2
Processing files 0 thru 3 in groups of 5...
0 , 
Processing files 4 thru 7 in groups of 5...
4 , 
Processing files 8 thru 11 in groups of 5...
8 , 
Processing files 12 thru 15 in groups of 5...
12 , 
Processing files 16 thru 19 in groups of 5...
16 , 
Processing files 20 thru 19 in groups of 5...


### DISK PART 3
Processing files 0 thru 3 in groups of 5...
0 , 
Processing files 4 thru 7 in groups of 5...
4 , 
Processing files 8 thru 11 in groups of 5...
8 , 
Processing files 12 thru 15 in groups of 5...
12 , 
Processing files 16 thru 19 in groups of 5...
16 , 
Processing files 20 thru 19 in groups of 5...


### DISK PART 4
Processing files 0 thru 3 in groups

In [ ]:
folder_path = f'{main_path}/LB/covis/{matrix_name}'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'{matrix_name}/*')
[shutil.copy(file, f'{main_path}/LB/covis/{matrix_name}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_5.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_4.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_2.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_3.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_12.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_18.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_13.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_9.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/LB/covis/test_covis_matrix/test_covis_matrix_6.parquet',
 '/content/drive/Othercomputers/Mac/Jup.Not

# Создание признаков test_candidates

In [ ]:
test_candidates_sessions = pd.read_parquet(
    f'{main_path}/LB/data/test_candidates_sessions.parquet')
train_main = pd.read_parquet(f'{main_path}/main_dataframes/train_main.parquet')

# Данные для item признаков
for_features_df_items = pd.concat([test_candidates_sessions, train_main])
for_features_df_items.sort_values(by='session', inplace=True)
for_features_df_items['ts'] = pd.to_datetime(for_features_df_items['ts'],
                                             unit='s')

# Данные для session, session-item признаков
for_features_df_sessions = test_candidates_sessions
for_features_df_sessions.sort_values(by='session', inplace=True)
for_features_df_sessions['ts'] = pd.to_datetime(for_features_df_sessions['ts'],
                                                unit='s')

del test_candidates_sessions, train_main

## Признаки товаров

**База 6 признаков**

In [ ]:
lv_item_features = for_features_df_items.groupby('aid').agg({
    'aid': 'count',
    'session': 'nunique',
    'type': 'mean'
})
lv_item_features.columns = [
    'item_quantity_in_df', 'sessions_w_item', 'mean_item_type'
]

In [ ]:
lv_item_features.item_quantity_in_df = lv_item_features.item_quantity_in_df.astype(
    np.int32)
lv_item_features.sessions_w_item = lv_item_features.sessions_w_item.astype(
    np.int32)
lv_item_features.mean_item_type = lv_item_features.mean_item_type.astype(
    np.float32)

In [ ]:
lv_item_features.reset_index(inplace = True)
lv_item_features.aid = lv_item_features.aid.astype(np.int32)

In [ ]:
lv_item_features.to_parquet(
    f'{main_path}/LB/features/test/base_item_features.parquet')

### Доп признаки

In [ ]:
for_features_df = pl.DataFrame({
    'session':
    pl.Series(for_features_df_items['session']),
    'aid':
    pl.Series(for_features_df_items['aid']),
    'type':
    pl.Series(for_features_df_items['type']),
    'ts':
    pl.Series(for_features_df_items['ts'])
})

In [ ]:
def get_time_diff(group) -> dict:
    '''
    Вычисляет временные разницы между различными типами событий в группе.

    Parameters:
    - group (DataFrame): Группа данных.

    Returns:
    - dict: Словарь с временными разницами.
    '''
    # Получаем минимальные временные метки для каждого типа события
    click_ts_min = group.filter(pl.col('type') == 0).select('ts').min().item()
    cart_ts_min = group.filter(pl.col('type') == 1).select('ts').min().item()
    ord_ts_min = group.filter(pl.col('type') == 2).select('ts').min().item()

    # Вычисляем временные разницы и преобразуем их в секунды
    if click_ts_min is not None and cart_ts_min is not None and \
            click_ts_min <= cart_ts_min:
        click_to_cart_t = int((cart_ts_min - click_ts_min).total_seconds())
    else:
        click_to_cart_t = -1

    if click_ts_min is not None and ord_ts_min is not None and \
            click_ts_min <= ord_ts_min:
        click_to_ord_t = int((ord_ts_min - click_ts_min).total_seconds())
    else:
        click_to_ord_t = -1

    # Возвращаем результат в виде словаря
    return {
        'aid': group['aid'][0],
        'cl_to_crt_med_time': click_to_cart_t,
        'cl_to_ord_med_time': click_to_ord_t
    }

In [ ]:
def most_active_time_(group) -> pl.DataFrame:
    """
        Определяет активное время на основе столбца 'ts' в группе данных.

        :params group: DataFrame с данными.
        :return (pl.DataFrame): Значение времени.
        """
    group = group.with_columns(pl.col('ts').dt.hour())

    def get_daytime(ts_value):
        if (ts_value >= 23) & (ts_value < 4):
            return 0  #ночь
        elif (ts_value >= 4) & (ts_value < 11):
            return 1  #утро
        elif (ts_value >= 11) & (ts_value < 15):
            return 2  #день
        else:
            return 3  #вечер

    most_active_t = group.with_columns(
        pl.col('ts').apply(get_daytime).alias('daytime'))
    most_active_t = most_active_t.with_columns(
        pl.median('daytime').alias('most_active_time').cast(pl.Int8))

    return most_active_t

In [ ]:
def create_features(group: pl.DataFrame) -> pl.DataFrame:
    """
    Создает признаки на основе различий во времени и активности пользователя
    для заданной группы данных.

    Параметры:
    - group: pandas.DataFrame
        Входной DataFrame, содержащий сессии пользователей и их взаимодействия.

    Возвращает:
    - pl.DataFrame
        DataFrame с уникальными значениями aid, включая медианные времена перехода
        от "click" к "cart" и от "click" к "order", а также самое активное время суток.
    """

    # Вычисление медианных времен для перехода от "click" к "cart" для каждой сессии.
    click_to_cart_times = (group.groupby('session').agg(
        **get_time_diff(group)).median().select(
            [pl.col('aid').cast(pl.Int32),
             pl.col('cl_to_crt_med_time')]))

    # Вычисление медианных времен для перехода от "click" к "order" для каждой сессии.
    click_to_ord_times = (group.groupby('session').agg(
        **get_time_diff(group)).median().select(
            [pl.col('aid').cast(pl.Int32),
             pl.col('cl_to_ord_med_time')]))

    # Вычисление времени суток наибольшей активности для каждого aid.
    most_active_time = most_active_time_(group) \
                        .select([pl.col('aid').cast(pl.Int32),
                                 pl.col('most_active_time')])

    # Создание DataFrame с уникальными aid, включающий медианные времена
    # и время суток наибольшей активности.
    return (
        group.select('aid').unique().join(
            click_to_cart_times, on='aid')  # Медианное время в секундах,
        # спустя которое пользователи переходят от "click" к "cart"  для данного aid.
        .join(click_to_ord_times, on='aid')  # Медианное время в секундах,
        # спустя которое пользователи переходят от "click" к "order" для данного aid.
        .join(most_active_time, on='aid')  # Время суток, в которые наиболее
        # активно взаимодействуют с этим aid.
        .unique())

In [ ]:
num_groups = len(for_features_df_items["aid"].unique().to_list())

with tqdm(total=num_groups) as pbar:
    dop_item_features = (for_features_df.groupby('aid').apply(
        w_pbar(pbar, create_features)))

100%|██████████| 1855603/1855603 [2:23:04<00:00, 216.17it/s]


In [ ]:
# Вычисление дополнительных признаков
first_aids = for_features_df.groupby('session').apply(lambda x: x.filter(
    pl.col('ts') == x.select('ts').min()).select(['session', 'aid']))
print(1)

last_aids = for_features_df.groupby('session').apply(lambda x: x.filter(
    pl.col('ts') == x.select('ts').max()).select(['session', 'aid']))
print(2)

aid_posit_percents = (for_features_df.join(first_aids, on='session').rename({
    'aid_right':
    'first_aid'
}).join(last_aids, on='session').rename({'aid_right': 'last_aid'}))
print(3)
aid_posit_percents = aid_posit_percents.drop(['ts', 'type']).unique()

In [ ]:
def aid_percents(group: pl.DataFrame) -> pl.DataFrame:
    """
    Рассчитывает процент сессий, в которых 'aid' является первым и последним aid.

    Параметры:
    - group (pl.DataFrame): Группа данных.

    Возвращает:
    - pl.DataFrame: DataFrame, содержащий 'aid', процент первого aid и процент последнего aid.
    """
    # Рассчитываем процент сессий, в которых 'aid' является первым aid
    first_aid_p = (
        group.filter(pl.col('aid') == pl.col('first_aid')).select(pl.count()) /
        group.select(pl.col('session').unique().count())).item()

    # Рассчитываем процент сессий, в которых 'aid' является последним aid
    last_aid_p = (
        group.filter(pl.col('aid') == pl.col('last_aid')).select(pl.count()) /
        group.select(pl.col('session').unique().count())).item()

    # Создаем DataFrame с 'aid', процентом первого aid и процентом последнего aid
    result_df = pl.DataFrame({
        'aid': group['aid'][0],
        'first_aid_percent': first_aid_p,
        'last_aid_percent': last_aid_p
    })

    return result_df

In [ ]:
# Вычисление доп признаков 'first_aid_percent', 'last_aid_percent'
num_groups = aid_posit_percents.get_column("aid").unique().len()

with tqdm(total=num_groups) as pbar:
    aid_posit_percents = aid_posit_percents \
                        .groupby('aid') \
                        .apply(w_pbar(pbar,aid_percents))

aid_posit_percents = aid_posit_percents.with_columns(
    pl.col('aid').cast(pl.Int32))

In [ ]:
dop_item_features = dop_item_features.join(aid_posit_percents, on = 'aid')

In [ ]:
dop_item_features.write_parquet(
    file=f'{main_path}/lv/ver_6/dop_item_features.parquet')

## Признаки сессий

**База 6 признаков**

In [ ]:
lv_user_features = for_features_df_sessions.groupby('session').agg({
    'session':
    'count',
    'aid':
    'nunique',
    'type':
    'mean'
})
lv_user_features.columns = [
    'session_quantity_in_df', 'items_in_session', 'mean_session_type'
]

In [ ]:
lv_user_features.session_quantity_in_df = lv_user_features.session_quantity_in_df.astype(
    np.int32)
lv_user_features.items_in_session = lv_user_features.items_in_session.astype(
    np.int32)
lv_user_features.mean_session_type = lv_user_features.mean_session_type.astype(
    np.float32)

In [ ]:
lv_user_features.reset_index(inplace=True)

In [ ]:
lv_user_features.session = lv_user_features.session.astype(np.int32)

In [ ]:
lv_user_features.to_parquet(
    f'{main_path}/LB/features/test/base_user_features.parquet')

### Доп признаки

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [ ]:
import cudf
import cupy as cp
from numba import cuda

In [ ]:
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7c2f6dacc0e0 to Device at 0x7c2f6dcb9c00>

In [ ]:
cu_for_features_df = cudf.DataFrame(for_features_df_sessions)

In [ ]:
chunk_size = 500000

chunks = [
    cu_for_features_df[i:i + chunk_size]
    for i in range(0, len(cu_for_features_df), chunk_size)
]

In [ ]:
# Определите структуру и названия столбцов для итогового DataFrame
columns = [
    'session', 'session_len', 'last_aid_type_0', 'last_aid_type_1',
    'last_aid_type_2', 'mn_time_b/w_aids', 'freq_per_clicks', 'freq_per_carts',
    'freq_per_orders', 'type_counts_clicks', 'type_counts_carts',
    'type_counts_orders', 'ratio_ses_clicks', 'ratio_ses_carts',
    'ratio_ses_orders', 'ses_aid_duplic_rate'
]

# Создайте итоговый DataFrame с пустой структурой
dop_ses_fts_df = cudf.DataFrame(columns=columns)

for chunk in tqdm(chunks):

    chunk = chunk.sort_values(by='session')
    # Создание признака 'session_len' (длина каждой сессии в минутах)
    session_lengths = round((chunk.groupby('session')['ts'].apply(lambda x: (
        x.max() - x.min())).astype('timedelta64[s]').astype('int64') / 60), 0)
    chunk['session_len'] = chunk['session'].map(session_lengths).astype(
        np.int32)

    # Создание признака 'last_aid_type' (последнее действие каждого типа в каждой сесси
    last_aid_type_0 = chunk[chunk['type'] == 0].groupby(
        ['session'])['aid'].transform('last')
    chunk['last_aid_type_0'] = last_aid_type_0
    chunk['last_aid_type_0'] = chunk['last_aid_type_0'].fillna(0)
    last_aid_type_1 = chunk[chunk['type'] == 1].groupby(
        ['session'])['aid'].transform('last')
    chunk['last_aid_type_1'] = last_aid_type_1
    chunk['last_aid_type_1'] = chunk['last_aid_type_1'].fillna(0)
    last_aid_type_2 = chunk[chunk['type'] == 2].groupby(
        ['session'])['aid'].transform('last')
    chunk['last_aid_type_2'] = last_aid_type_2
    chunk['last_aid_type_2'] = chunk['last_aid_type_2'].fillna(0)

    # 'mn_time_b/w_aids' Среднее время между действиями в сессии
    dfff = chunk.to_pandas()
    dfff = dfff.sort_values(['session', 'ts'])
    dfff['time_diff'] = dfff.groupby('session')['ts'].diff()
    average_time_between_aids = dfff.groupby(
        'session')['time_diff'].mean().astype('timedelta64[s]')
    dfff['mn_time_b/w_aids'] = dfff['session'].map(average_time_between_aids)
    dfff['mn_time_b/w_aids'] = dfff['mn_time_b/w_aids'].fillna(0)
    dfff['mn_time_b/w_aids'] = round(dfff['mn_time_b/w_aids'],
                                     0).astype(np.int32)
    dfff.drop(columns=['time_diff'], inplace=True)
    chunk = cudf.DataFrame(dfff)

    # 'type_counts_ses' Кол-во действий кажд.типа в session
    chunk['type_counts_clicks'] = chunk['session'].map(
        chunk[chunk['type'] == 0].groupby('session')['aid'].count())
    chunk['type_counts_carts'] = chunk['session'].map(
        chunk[chunk['type'] == 1].groupby('session')['aid'].count())
    chunk['type_counts_orders'] = chunk['session'].map(
        chunk[chunk['type'] == 2].groupby('session')['aid'].count())

    # freq_per_type' Частота кликов/ картов/ ордеров(кол-во/длина сессии в мин) в session
    chunk[
        'freq_per_clicks'] = chunk['type_counts_clicks'] / chunk['session_len']
    chunk['freq_per_carts'] = chunk['type_counts_carts'] / chunk['session_len']
    chunk[
        'freq_per_orders'] = chunk['type_counts_orders'] / chunk['session_len']

    def ratio_ses_types(series_):
        len_ = chunk['session'].map(chunk.groupby('session')['aid'].size())
        return series_ / len_

    # 'type_ratio_ses' aid clicks/orders/carts ratio в сессии
    chunk['ratio_ses_clicks'] = ratio_ses_types(chunk['type_counts_clicks'])
    chunk['ratio_ses_carts'] = ratio_ses_types(chunk['type_counts_carts'])
    chunk['ratio_ses_orders'] = ratio_ses_types(chunk['type_counts_orders'])

    # 'ses_aid_duplic_rate' Были ли в данной сессии повторяющиеся aid, и если да, то сколько раз.
    unique_aid_count = chunk.groupby(['session'])['aid'].nunique()
    total_aid_count = chunk.groupby(['session'])['aid'].count()
    chunk['ses_aid_duplic_rate'] = chunk['session'].map(unique_aid_count /
                                                        total_aid_count)

    chunk = chunk.drop(columns=['ts', 'type', 'aid'])

    dop_ses_fts_df = cudf.concat([dop_ses_fts_df, chunk])

  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/cudf/core/groupby/groupby.py:1273: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 146086 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(
 17%|█▋        | 1/6 [01:39<08:17, 99.58s/it]/usr/local/lib/python3.10/dist-packages/cudf/core/groupby/groupby.py:1273: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 155896 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(
 33%|███▎      | 2/6 [03:27<06:58, 104.58s/it]/usr/local/lib/python3.10/dist-packages/cudf/core/groupby/groupby.py:1273: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 155898 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(
 50%|█████     | 3/6 [05:16<05:19, 106.54s/it]/usr/local/lib/python3.10/dist-packages/cudf/core/groupby/groupby.py:1273: R

In [ ]:
dop_ses_fts_df

,session,session_len,last_aid_type_0,last_aid_type_1,last_aid_type_2,mn_time_b/w_aids,freq_per_clicks,freq_per_carts,freq_per_orders,type_counts_clicks,type_counts_carts,type_counts_orders,ratio_ses_clicks,ratio_ses_carts,ratio_ses_orders,ses_aid_duplic_rate
0,12899780,1,582732,0,0,58,2.000000,<NA>,<NA>,2,<NA>,<NA>,1.0,<NA>,<NA>,1.0
1,12899780,1,582732,0,0,58,2.000000,<NA>,<NA>,2,<NA>,<NA>,1.0,<NA>,<NA>,1.0
2,12899781,944,199008,0,0,14155,0.005297,<NA>,<NA>,5,<NA>,<NA>,1.0,<NA>,<NA>,0.8
3,12899781,944,199008,0,0,14155,0.005297,<NA>,<NA>,5,<NA>,<NA>,1.0,<NA>,<NA>,0.8
4,12899781,944,199008,0,0,14155,0.005297,<NA>,<NA>,5,<NA>,<NA>,1.0,<NA>,<NA>,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907450,14571534,0,272221,0,0,7,inf,Inf,<NA>,1,1,<NA>,0.5,0.5,<NA>,0.5
2907451,14571534,0,0,272221,0,7,inf,Inf,<NA>,1,1,<NA>,0.5,0.5,<NA>,0.5
2907452,14571539,0,317311,0,0,0,inf,<NA>,<NA>,1,<NA>,<NA>,1.0,<NA>,<NA>,1.0
2907453,14571547,0,1546409,0,0,0,inf,<NA>,<NA>,1,<NA>,<NA>,1.0,<NA>,<NA>,1.0


In [ ]:
dop_ses_fts_df.to_parquet(f'{main_path}/LB/features/test/dop_user_features.parquet')

## признаки Сессии - товары

**Косинусное расстояние_1/3 часть**

In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import polars as pl

In [ ]:
data_pl = pl.DataFrame({
    'session': pl.Series(for_features_df_sessions['session']),
    'aid': pl.Series(for_features_df_sessions['aid'])
})

In [ ]:
sentences_df = data_pl.groupby('session').agg(pl.col('aid').alias('sentence'))

In [ ]:
%%time
# Создание "предложений" для каждого пользователя
sentences = sentences_df['sentence'].to_list()

CPU times: user 1.28 s, sys: 91.4 ms, total: 1.38 s
Wall time: 1.31 s


In [ ]:
%%time
# Обучение модели Word2Vec
model = Word2Vec(sentences=sentences, vector_size=32, min_count=1, workers=4)

CPU times: user 1min, sys: 369 ms, total: 1min 1s
Wall time: 28.3 s


In [ ]:
# Вычисление векторных представлений для пользователей
user_vectors = {}
for row in tqdm(sentences_df.iter_rows()):

    session = row[0]
    history = row[1]
    user_vector = np.mean([model.wv[item] for item in history if item in model.wv], axis=0)
    user_vectors[session] = user_vector

921704it [00:21, 43187.03it/s]


In [ ]:
def compute_cosine_similarity(session: str, aid: str) -> float:
    """
    Вычисляет косинусное сходство между вектором пользователя и вектором товара.

    :param session: Уникальный идентификатор сеанса пользователя.

    :param aid: Уникальный идентификатор товара (item).

    :return: Косинусное сходство между векторами пользователя и товара.
    """
    # Получаем вектор пользователя из предварительно вычисленных векторов.
    user_vector: np.ndarray = user_vectors[session]

    # Получаем вектор товара из модели Word2Vec или используем нулевой вектор, если товар отсутствует в модели.
    item_vector: np.ndarray = model.wv[aid] if aid in model.wv else np.zeros(model.vector_size)

    # Вычисляем косинусное сходство между векторами пользователя и товара.
    cosine_sim: float = cosine_similarity([user_vector], [item_vector])[0][0]

    return cosine_sim

num_groups = data_pl.get_column("session").len()
data_pl = data_pl.unique(subset=['session', 'aid'])

with tqdm(total=num_groups) as pbar:

    data_pl_f = data_pl.with_columns(
        pl.struct(['session','aid']) \
          .apply(w_pbar(pbar, lambda x: compute_cosine_similarity(x['session'], x['aid']))).alias('cosine_similarity')
    )

In [ ]:
data_pl_f.write_parquet(f'{main_path}/LB/features/test/user_item_features.parquet')

# Создание датасета test_candidates

## Создание списка test_candidates

In [ ]:
cand_n = 50

In [ ]:
test_candidates_sessions = pd.read_parquet(
    f'{main_path}/LB/data/test_candidates_sessions.parquet')

In [ ]:
covis_matrix = read_covis_to_dict(name='test_covis_matrix', n=20)

100%|██████████| 20/20 [00:39<00:00,  1.99s/it]


In [ ]:
top_pop_aids = set(
    test_candidates_sessions['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
# Преобразование датафрейма в словарь
test_dict = test_candidates_sessions.sort_values(
    ["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

In [ ]:
# Создание списка самих кандидатов для будушего тест.датасета
labels, covis_features = create_labels(n=cand_n,
                                       inp_dict=test_dict,
                                       covis=covis_matrix,
                                       top_pop_aids=top_pop_aids,
                                       cov_f=True)

100%|██████████| 921704/921704 [02:03<00:00, 7457.85it/s]


Словарь с признаком-рангом по ковиз-матрице сохранен в : /content/drive/Othercomputers/Mac/Jup.Notebook/LB/features/test/covis_features.pickle


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|██████████| 921704/921704 [00:00<00:00, 2219483.11it/s]


In [ ]:
test_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
test_candidates_['session'] = test_candidates_['session'].astype(np.int32)

In [ ]:
# Сохранение списка самих кандидатов для будущего создания тест.датасета
folder_path = f'{main_path}/LB/test_candidates'
os.makedirs(folder_path, exist_ok=True)

partt = len(test_candidates_) // 5

for batch in tqdm(range(5), desc='part'):

    start = partt * batch
    end = partt + start
    print(start, end)

    if batch == 4:
        batch_ = test_candidates_.loc[start:end]
    else:
        batch_ = test_candidates_.loc[start:end - 1]

    batch_ = batch_.explode('labels')

    batch_['labels'] = batch_['labels'].astype(np.int32)

    batch_.to_parquet(f'{folder_path}/test_candidates_{batch}.parquet')

part:   0%|          | 0/5 [00:00<?, ?it/s]

0 184340


part:  20%|██        | 1/5 [00:02<00:10,  2.50s/it]

184340 368680


part:  40%|████      | 2/5 [00:04<00:07,  2.50s/it]

...

part:  60%|██████    | 3/5 [00:07<00:04,  2.50s/it]

553020 737360


part:  80%|████████  | 4/5 [00:10<00:02,  2.51s/it]

737360 921700


part: 100%|██████████| 5/5 [00:12<00:00,  2.52s/it]


In [ ]:
del test_candidates_

## Преобразование covis - features

Преобразование pickle признака-ранга по матрице ко-посещений в parquet

In [ ]:
features_df = (pd.DataFrame.from_dict([
    (session_id, aid, rank) for session_id, items in covis_features.items()
    for aid, rank in items.items()
]))

In [ ]:
features_df.columns = ['session', 'aid', 'rank']
features_df['session'] = features_df['session'].astype(np.int32)
features_df['aid'] = features_df['aid'].astype(np.int32)
features_df['rank'] = features_df['rank'].astype(np.uint8)

In [ ]:
folder_path = f'{main_path}/LB/features/test/covis_features'
os.makedirs(folder_path, exist_ok=True)

features_df.reset_index(drop=True, inplace=True)

partt = len(features_df) // 5

for batch in tqdm(range(5), desc='part'):

    start = partt * batch
    end = partt * (batch + 1)
    batch_ = features_df.loc[start:end]

    batch_.to_parquet(f'{folder_path}/covis_features_{batch}.parquet')

part: 100%|██████████| 5/5 [00:02<00:00,  2.09it/s]


## Создание признаков  популярности товара по типам

In [ ]:
for_features_df = test_candidates_sessions.drop(columns=['ts'])

In [ ]:
# Сгруппируем исходный DataFrame по столбцу 'type'
grouped = for_features_df.groupby('type')

id_2_type = {0: 'clicks', 1: 'carts', 2: 'orders'}

# Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
for name, group in tqdm(grouped):
    # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
    aid_counts = group['aid'].value_counts().reset_index()
    aid_counts.columns = ['aid', 'count']

    # Добавим столбец с рангом, используя функцию rank()
    aid_counts['rank_pops'] = aid_counts['count'].rank(ascending=False,
                                                       method='dense')
    aid_counts.drop(columns=['count'], inplace=True)

    # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
    group = group.drop(columns=['session', 'type']).drop_duplicates(
        subset=['aid'])
    merged_group = pd.merge(group, aid_counts, on='aid', how='left')
    merged_group.fillna(-1, inplace=True)
    merged_group['rank_pops'] = merged_group['rank_pops'].astype(np.int32)
    merged_group.reset_index(inplace=True, drop=True)

    merged_group.to_parquet(
        f'{main_path}/LB/features/test/pop_features_{id_2_type[name]}.parquet')

100%|██████████| 3/3 [00:00<00:00,  4.05it/s]


## Добавление признаков к test_candidates

In [ ]:
base_item_features = pd.read_parquet(
    f'{main_path}/LB/features/test/base_item_features.parquet')
base_user_features = pd.read_parquet(
    f'{main_path}/LB/features/test/base_user_features.parquet')
dop_user_features = pd.read_parquet(
    f'{main_path}/LB/features/test/dop_user_features.parquet')
dop_item_featues = pd.read_parquet(
    f'{main_path}/LB/features/test/dop_item_features.parquet')
user_item_features = pd.read_parquet(
    f'{main_path}/LB/features/test/user_item_features.parquet')

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/LB/test_candidates/*')
file_list_covis_f = glob.glob(f'{main_path}/LB/features/test/covis_features/*')
file_list_pops = glob.glob(f'{main_path}/LB/features/test/pop_features_*.parquet')

file_list.sort()
file_list_covis_f.sort()
file_list_pops.sort()

In [ ]:
# Цикл для чтения и объединения файлов
for iter_numb, (file,
                file_2) in tqdm(enumerate(zip(file_list, file_list_covis_f)),
                                desc=f'parts'):

    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)
    covis_f = pd.read_parquet(file_2)

    df_ = df_.rename(columns={'labels': 'aid'})
    # Базовые user, item features
    df_ = df_.merge(base_item_features, on=['aid'], how='left').fillna(-1)
    df_ = df_.merge(base_user_features, on=['session'], how='left').fillna(-1)
    # User-item features
    df_ = df_.merge(user_item_features, on=['session', 'aid'],
                    how='left').fillna(-1)
    # Доп. user features
    df_ = df_.merge(dop_user_features[[
        'session', 'session_len', 'mean_time_bw_aids', 'ses_aid_duplic_rate'
    ]],
                    on=['session'],
                    how='left').fillna(-1)
    df_ = df_.merge(covis_f, on=['session', 'aid'], how='left').fillna(-1)
    #Доп. item features
    df_ = df_.merge(dop_item_featues, on=['aid'], how='left').fillna(-1)
    print('Done 1/3')

    for (type_, file_path_, index) in tqdm(
        (zip(['carts', 'clicks', 'orders'], file_list_pops, [1, 0, 2])),
            desc=f'{iter_numb}_types'):

        # Доп. item features - по type
        columns_ = ['aid', 'rank_pops']
        pop_features = pd.read_parquet(file_path_)
        df_m = df_.merge(pop_features[columns_], on=['aid'],
                         how='left').fillna(-1)
        print('Done 2/5')
        #Доп. user features - по type
        df_m = df_m.merge(
            dop_user_features[['session', f'last_aid_type_{index}']],
            on=['session'],
            how='left').fillna(-1)
        print('Done 3/5')
        columns_user_item = [
            'session', f'freq_per_{type_}', f'ratio_ses_{type_}',
            f'type_counts_{type_}'
        ]
        df_m = df_m.merge(dop_user_features[columns_user_item],
                          on=['session'],
                          how='left').fillna(-1)
        print('Done 4/5')

        folder_path = f'{main_path}/LB/test_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        df_m.to_parquet(
            f"{folder_path}/test_candidates_{type_}_{iter_numb}.parquet")

parts: 0it [00:00, ?it/s]

Done 1/3



0_types: 0it [00:00, ?it/s]

Done 2/5
Done 3/5
Done 4/5



0_types: 1it [00:14, 14.90s/it]

...


4_types: 1it [00:14, 14.75s/it]

Done 2/5
Done 3/5
Done 4/5



4_types: 2it [00:29, 14.72s/it]

Done 2/5
Done 3/5
Done 4/5



4_types: 3it [00:44, 14.82s/it]
parts: 5it [05:47, 69.42s/it]


# Предсказание

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.9 MB/s eta 0:00:00


In [ ]:
import catboost
from catboost import CatBoostRanker, Pool

In [ ]:
import os
os.environ['CATBOOST_GPU'] = '1'

In [ ]:
params_name = 'catb_tun'

**Предсказание на тесте**

In [ ]:
test_predictions_full = pd.DataFrame()

# Прохождение по каждому типу кандидатов
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    path = f"{main_path}/LB/test_candidates_{typee}/*"
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    # Подготовка частей данных для предсказания
    FEATURES = test_candidates.columns[2:]
    dtest = Pool(data=test_candidates[FEATURES])
    preds = np.zeros(len(test_candidates))

    # Предсказание и его усреднение по 5 фолдам
    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(
            f'{main_path}/models/{params_name}/{params_name}_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    # Отбор топ-20 предсказаний для сессии и доп трансформация
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    test_predictions_full = pd.concat([test_predictions_full, lv_predictions],
                                      ignore_index=True)
    del lv_predictions, preds, test_candidates

100%|██████████| 5/5 [00:11<00:00,  2.31s/it]

100%|██████████| 5/5 [00:22<00:00,  4.42s/it]

100%|██████████| 5/5 [00:22<00:00,  4.57s/it]

type: 100%|██████████| 3/3 [09:54<00:00, 198.00s/it]


**Предсказание на трейне** \
Получаю предсказания на тренировочном датасете  
 кандидатов, на котором обучалась модель

In [ ]:
train_predictions_full = pd.DataFrame()

# Прохождение по каждому типу кандидатов
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    path = f"{main_path}/tr/ver_6/tr_candidates_{typee}/*"
    train_candidates = read_parquets(path)
    train_candidates = train_candidates.sort_values('session')
    train_candidates = train_candidates.reset_index(drop=True)

    # Подготовка частей данных для предсказания
    FEATURES = train_candidates.columns[2:-1]
    dtest = Pool(data=train_candidates[FEATURES])
    preds = np.zeros(len(train_candidates))

    # Предсказание и его усреднение по 5 фолдам
    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(
            f'{main_path}/models/{params_name}/{params_name}_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    # Отбор топ-20 предсказаний для сессии и доп трансформация
    lv_predictions = train_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    train_predictions_full = pd.concat(
        [train_predictions_full, lv_predictions], ignore_index=True)
    del lv_predictions, preds, train_candidates

type: 100%|██████████| 3/3 [02:59<00:00, 59.90s/it]


# Оценка метрики

**На тесте**

In [ ]:
id2type_name = 'id2type.pkl'

In [ ]:
test_labels = pd.read_parquet(f'{main_path}/LB/data/test_labels.parquet')

In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

In [ ]:
print('Model score :', metric_eval(test_predictions_full, test_labels,
                                   id2type))

Model score : 0.5281948462262794


**На трейне**

In [ ]:
# train_predictions_full.to_parquet(f'{main_path}/LB/train_predictions_full.parquet')
# train_predictions_full = pd.read_parquet(f'{main_path}/LB/train_predictions_full.parquet')

In [ ]:
# В роли test_labels испольую таргеты, созданные для обучения на этом тренировочном
# датасете (на котором я выше предсказался)
test_labels = pd.read_parquet(f'{main_path}/s/targets.parquet')

In [ ]:
print('Model score :', metric_eval(train_predictions_full, test_labels,
                                   id2type))

Model score : 0.52907483652647848


In [ ]:
print(abs(0.52907483652647848 - 0.5281948462262794)/0.5281948462262794*100, '%')

0.1666033484586558 %


**Модель не переобучилась, разница в метрике между трейн и тест = 0.16%**

# Вывод

Я проверил производительность обученной ранее модели на отложенном оригинальном тестовом  
датасете test_main
- метрика получилась схожей с метрикой, полученной на тестовой части, взятой из   
послед.недели train_main  
- переобучения не обнаружилось